In [69]:
# Import useful packages
import os
import time
import pandas as pd
import glob
import numpy as np

# For loading/querying postgres
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import connect

In [70]:
# Create SQL engine to load data to postgres for central usage
from sqlalchemy import create_engine
dbname = '697_temp'
user = 'postgres'
password = 'poRter!5067'
host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com'

engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

### Loading Open Positions to Postgres

In [71]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Open_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Open_Positions_Data\\MF_MLP.csv']

In [72]:
# create temp_df to test loading; pick model not already in table
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
0,Positions_2019-10-01,2019-10-01,WMT,1726,198464.585948,114.985276,200558.811639,116.198616,0.0,MF_MLP
1,Positions_2019-10-01,2019-10-01,MA,632,170062.080208,269.085570,189199.807972,299.366785,0.0,MF_MLP
2,Positions_2019-10-01,2019-10-01,CVX,1315,141241.337962,107.407862,146882.146841,111.697450,0.0,MF_MLP
3,Positions_2019-10-01,2019-10-01,NFLX,309,82901.610283,268.290001,101249.258537,327.667503,0.0,MF_MLP
4,Positions_2019-10-01,2019-10-01,ABBV,912,61986.386626,67.967529,75257.748234,82.519461,0.0,MF_MLP


In [73]:
temp_df.columns

Index(['key', 'Date', 'Ticker', 'Quantity', 'Basis', 'Purchase Price',
       'Current Value', 'Last', '% Gain', 'model'],
      dtype='object')

In [74]:
# Initial dataframe needs to create a new table
#temp_df.to_sql('open_positions', engine)

In [75]:
# Append data to table
temp_df.to_sql('open_positions', engine, if_exists='append')

In [76]:
# Test with a load
def import_open_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM open_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['key', 'model'])
    return data
open_pos_df = import_open_positions().drop('index',axis=1)

open_pos_df[open_pos_df['model']=='MF_MLP']


,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
18347,Positions_2019-10-01,2019-10-01,WMT,1726,198464.585948,114.985276,200558.811639,116.198616,0.00,MF_MLP
18348,Positions_2019-10-01,2019-10-01,MA,632,170062.080208,269.085570,189199.807972,299.366785,0.00,MF_MLP
18349,Positions_2019-10-01,2019-10-01,CVX,1315,141241.337962,107.407862,146882.146841,111.697450,0.00,MF_MLP
18350,Positions_2019-10-01,2019-10-01,NFLX,309,82901.610283,268.290001,101249.258537,327.667503,0.00,MF_MLP
18351,Positions_2019-10-01,2019-10-01,ABBV,912,61986.386626,67.967529,75257.748234,82.519461,0.00,MF_MLP
...,...,...,...,...,...,...,...,...,...,...
23886,Positions_2021-06-30,2021-04-01,CVX,487,50646.537237,103.996996,50791.664108,104.294998,0.29,MF_MLP
23887,Positions_2021-06-30,2021-04-01,WFC,1007,39478.905825,39.204475,45252.062500,44.937500,14.62,MF_MLP
23888,Positions_2021-06-30,2021-04-01,MS,489,37966.028270,77.640140,44486.775187,90.975000,17.18,MF_MLP
23889,Positions_2021-06-30,2021-04-01,C,519,37479.716676,72.215254,36415.634485,70.164999,-2.84,MF_MLP


In [77]:
df = open_pos_df[open_pos_df['model']=='MF_MLP']

In [78]:
df

,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
18347,Positions_2019-10-01,2019-10-01,WMT,1726,198464.585948,114.985276,200558.811639,116.198616,0.00,MF_MLP
18348,Positions_2019-10-01,2019-10-01,MA,632,170062.080208,269.085570,189199.807972,299.366785,0.00,MF_MLP
18349,Positions_2019-10-01,2019-10-01,CVX,1315,141241.337962,107.407862,146882.146841,111.697450,0.00,MF_MLP
18350,Positions_2019-10-01,2019-10-01,NFLX,309,82901.610283,268.290001,101249.258537,327.667503,0.00,MF_MLP
18351,Positions_2019-10-01,2019-10-01,ABBV,912,61986.386626,67.967529,75257.748234,82.519461,0.00,MF_MLP
...,...,...,...,...,...,...,...,...,...,...
23886,Positions_2021-06-30,2021-04-01,CVX,487,50646.537237,103.996996,50791.664108,104.294998,0.29,MF_MLP
23887,Positions_2021-06-30,2021-04-01,WFC,1007,39478.905825,39.204475,45252.062500,44.937500,14.62,MF_MLP
23888,Positions_2021-06-30,2021-04-01,MS,489,37966.028270,77.640140,44486.775187,90.975000,17.18,MF_MLP
23889,Positions_2021-06-30,2021-04-01,C,519,37479.716676,72.215254,36415.634485,70.164999,-2.84,MF_MLP


### Loading track records to postgres

In [79]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("track_record/*.csv"):
    txtfiles.append(file)
    
txtfiles

['track_record\\MF_MLP.csv']

In [80]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

,Date,Value,Val_ex_cash,model
0,2019-10-01,1000000.000000,997480.856687,MF_MLP
1,2019-10-02,980878.978526,978359.835213,MF_MLP
2,2019-10-03,976557.625012,974038.481700,MF_MLP
3,2019-10-04,992352.700279,989833.556966,MF_MLP
4,2019-10-07,995624.682577,993105.539264,MF_MLP


In [81]:
# Initial dataframe needs to create a new table
#temp_df.to_sql('track_record', engine)

In [82]:
# Append data to table
temp_df.to_sql('track_record', engine, if_exists='append')

In [83]:
# Test with a load
def import_track_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM track_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['Date', 'model'])
    return data
track_df = import_track_record().drop('index',axis=1)

track_df[track_df['model']=='MF_MLP']


,Date,Value,Val_ex_cash,model
936,2019-10-01,1.000000e+06,9.974809e+05,MF_MLP
937,2019-10-02,9.808790e+05,9.783598e+05,MF_MLP
938,2019-10-03,9.765576e+05,9.740385e+05,MF_MLP
939,2019-10-04,9.923527e+05,9.898336e+05,MF_MLP
940,2019-10-07,9.956247e+05,9.931055e+05,MF_MLP
...,...,...,...,...
1378,2021-06-24,1.568415e+06,1.567857e+06,MF_MLP
1379,2021-06-25,1.569243e+06,1.568685e+06,MF_MLP
1380,2021-06-28,1.574647e+06,1.574089e+06,MF_MLP
1381,2021-06-29,1.586247e+06,1.585689e+06,MF_MLP


### Loading cash records to postgres

In [84]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Cash_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Cash_Positions_Data\\MF_MLP.csv']

In [86]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df

,key,cash,model
0,cash_2019-10-01,2519.143313,MF_MLP
1,cash_2019-10-02,2519.143313,MF_MLP
2,cash_2019-10-03,2519.143313,MF_MLP
3,cash_2019-10-04,2519.143313,MF_MLP
4,cash_2019-10-07,2519.143313,MF_MLP
...,...,...,...
436,cash_2021-06-24,558.255325,MF_MLP
437,cash_2021-06-25,558.255325,MF_MLP
438,cash_2021-06-28,558.255325,MF_MLP
439,cash_2021-06-29,558.255325,MF_MLP


In [87]:
# Initial dataframe needs to create a new table
temp_df.to_sql('cash_record', engine, if_exists='append')

In [88]:
# Test with a load
def import_cash_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM cash_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
cash_df = import_cash_record().drop('index',axis=1)

cash_df.tail()


,key,cash,model
418,cash_2021-03-11,5.490783,RF Reg_target_7_rebal_7_2017-01-01
419,cash_2021-03-12,5.490783,RF Reg_target_7_rebal_7_2017-01-01
420,cash_2021-03-15,5.490783,RF Reg_target_7_rebal_7_2017-01-01
421,cash_2021-03-16,5.490783,RF Reg_target_7_rebal_7_2017-01-01
422,cash_2021-03-17,5.490783,RF Reg_target_7_rebal_7_2017-01-01


In [89]:
cash_df[cash_df['model']=='MF_MLP']

,key,cash,model
936,cash_2019-10-01,2519.143313,MF_MLP
937,cash_2019-10-02,2519.143313,MF_MLP
938,cash_2019-10-03,2519.143313,MF_MLP
939,cash_2019-10-04,2519.143313,MF_MLP
940,cash_2019-10-07,2519.143313,MF_MLP
...,...,...,...
1372,cash_2021-06-24,558.255325,MF_MLP
1373,cash_2021-06-25,558.255325,MF_MLP
1374,cash_2021-06-28,558.255325,MF_MLP
1375,cash_2021-06-29,558.255325,MF_MLP


### Loading Value Positions to postgres

In [90]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Val_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Val_Positions_Data\\MF_MLP.csv']

In [91]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.tail()

,key,value,model
436,val_2021-06-24,1.568415e+06,MF_MLP
437,val_2021-06-25,1.569243e+06,MF_MLP
438,val_2021-06-28,1.574647e+06,MF_MLP
439,val_2021-06-29,1.586247e+06,MF_MLP
440,val_2021-06-30,1.591528e+06,MF_MLP


In [92]:
# Initial dataframe needs to create a new table
temp_df.to_sql('val_positions', engine,if_exists='append')

In [93]:
# Test with a load
def import_val_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM val_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
val_df = import_val_positions().drop('index',axis=1)

val_df.tail()

,key,value,model
422,val_2021-03-11,129090.678412,RF Reg_target_7_rebal_7_2017-01-01
423,val_2021-03-12,130598.742099,RF Reg_target_7_rebal_7_2017-01-01
424,val_2021-03-15,132633.335128,RF Reg_target_7_rebal_7_2017-01-01
425,val_2021-03-16,132407.606240,RF Reg_target_7_rebal_7_2017-01-01
426,val_2021-03-17,132356.878907,RF Reg_target_7_rebal_7_2017-01-01


In [95]:
val_df[val_df['model'] == 'MF_MLP']

,key,value,model
573,val_2019-10-01,1.000000e+06,MF_MLP
655,val_2019-10-02,9.808790e+05,MF_MLP
737,val_2019-10-03,9.765576e+05,MF_MLP
819,val_2019-10-04,9.923527e+05,MF_MLP
901,val_2019-10-07,9.956247e+05,MF_MLP
...,...,...,...
1372,val_2021-06-24,1.568415e+06,MF_MLP
1373,val_2021-06-25,1.569243e+06,MF_MLP
1374,val_2021-06-28,1.574647e+06,MF_MLP
1375,val_2021-06-29,1.586247e+06,MF_MLP
